In [1]:
import cupy
import numpy as np
import pandas as pd

add_kernel = cupy.RawKernel(r'''
extern "C" __global__
void my_add(const float* x1, const float* x2, float* y) {
     int tid = blockDim.x * blockIdx.x + threadIdx.x;
     y[tid] = x1[tid] + x2[tid];
}
''', 'my_add')
x1 = cupy.arange(25, dtype=cupy.float32).reshape(5, 5)
x2 = cupy.arange(25, dtype=cupy.float32).reshape(5, 5)
y = cupy.zeros((5, 5), dtype=cupy.float32)
add_kernel((5,), (5,), (x1, x2, y))  # grid, block and arguments
y


array([[ 0.,  2.,  4.,  6.,  8.],
       [10., 12., 14., 16., 18.],
       [20., 22., 24., 26., 28.],
       [30., 32., 34., 36., 38.],
       [40., 42., 44., 46., 48.]], dtype=float32)

In [2]:
import math
import numpy as np
import chainer as ch
from chainer import backend
from chainer import backends
from chainer.backends import cuda
from chainer import Function, FunctionNode, gradient_check, report, training, utils, Variable
from chainer import datasets, initializers, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

import matplotlib
matplotlib.use('Agg')

In [3]:
mushroomsfile = 'mushrooms.csv'
data_array = np.genfromtxt(
    mushroomsfile, delimiter=',', dtype=str, skip_header=1)
for col in range(data_array.shape[1]):
    data_array[:, col] = np.unique(data_array[:, col], return_inverse=True)[1]

X = data_array[:, 1:].astype(np.float32)
Y = data_array[:, 0].astype(np.int32)[:, None]
train, test = datasets.split_dataset_random(
    datasets.TupleDataset(X, Y), int(data_array.shape[0] * .7))

In [4]:
train_iter = ch.iterators.SerialIterator(train, 100)
test_iter = ch.iterators.SerialIterator(
    test, 100, repeat=False, shuffle=False)

In [5]:
# Network definition
def MLP(n_units, n_out):
    layer = ch.Sequential(L.Linear(n_units), F.relu)
    model = layer.repeat(2)
    model.append(L.Linear(n_out))

    return model

In [6]:
model = L.Classifier(
    MLP(44, 1), lossfun=F.sigmoid_cross_entropy, accfun=F.binary_accuracy)

In [7]:
# Setup an optimizer
optimizer = ch.optimizers.SGD().setup(model)

In [8]:
# Create the updater, using the optimizer
# updater = training.StandardUpdater(train_iter, optimizer, device=-1)
updater = training.StandardUpdater(train_iter, optimizer, device=0)

In [9]:
# Set up a trainer
trainer = training.Trainer(updater, (50, 'epoch'), out='result')

In [10]:
# Evaluate the model with the test dataset for each epoch
# trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.Evaluator(test_iter, model, device=0))

In [11]:
# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.DumpGraph('main/loss'))

In [12]:
trainer.extend(extensions.snapshot(), trigger=(20, 'epoch'))

In [13]:
# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

In [14]:
# Save two plot images to the result dir
if extensions.PlotReport.available():
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                              'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(
            ['main/accuracy', 'validation/main/accuracy'],
            'epoch', file_name='accuracy.png'))

In [15]:
# Print selected entries of the log to stdout
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

In [16]:
#  Run the training
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.605074    0.525185              0.688947       0.735432                  1.05294       
2           0.490505    0.441535              0.774737       0.819137                  1.45402       
3           0.424657    0.385829              0.827895       0.86339                   1.82796       
4           0.380296    0.348908              0.855614       0.882189                  2.2282        
5           0.350409    0.318939              0.869649       0.894842                  2.59759       
6           0.325195    0.300947              0.879824       0.902842                  2.96831       
7           0.3077      0.278649              0.889474       0.910695                  3.33474       
8           0.291724    0.264808              0.893929       0.918295                  3.6997        
9           0.279132    0.252419              0.901052       0.920295               

In [31]:
x, t = test[np.random.randint(len(test))]

# xnp = np.array(x[None])
xcupy = cupy.array(x[None])
predict = model.predictor(xcupy).array

# predict = model.predictor(x[None]).array
predict = predict[0][0]

if predict >= 0:
    print('Predicted Poisonous, Actual ' + ['Edible', 'Poisonous'][t[0]])
else:
    print('Predicted Edible, Actual ' + ['Edible', 'Poisonous'][t[0]])



# x, t = test[np.random.randint(len(test))]

# predict = model.predictor(x[None]).array
# predict = predict[0][0]

# if predict >= 0:
#     print('Predicted Poisonous, Actual ' + ['Edible', 'Poisonous'][t[0]])
# else:
#     print('Predicted Edible, Actual ' + ['Edible', 'Poisonous'][t[0]])


Predicted Edible, Actual Edible
